#### **DATA 622 - Assignment 2 - Decision Trees Algorithms**

Author: Kory Martin     
Date: 3/18/2024


**Instructions:**

Pre-work:
- Read this blog: https://decizone.com/blog/the-good-the-bad-the-ugly-of-using-decision-trees which shows some of the issues with decision trees
- Choose a dataset from a source in Assignment #1, or another dataset of your choice.

Assignment work:
- Based on the latest topics presented, choose a dataset of your choice and create a Decision Tree where you can solve a classification problem and predict the outcome of a particular feature or detail of the data used.
- Switch variables* to generate 2 decision trees and compare the results. Create a random forest and analyze the results.
- Based on real cases where desicion trees went wrong, and 'the bad & ugly' aspects of decision trees (https://decizone.com/blog/the-good-the-bad-the-ugly-of-using-decision-trees), how can you change this perception when using the decision tree you created to solve a real problem?

Deliverable:
- Essay (minimum 500 word document)
- Write a short essay explaining your analysis, and how you would address the concerns in the blog (listed in pre-work)
- Exploratory Analysis using R or Python (submit code + errors + analysis as notebook or copy/paste to document)


Note:
1. We are trying to train 2 different decision trees to compare bias and variance - so swicth the features used for the first node (split) to force a different decision tree (How did the performance change?)
2. You will create 3 models: 2 x decision trees (to compare variance) and a random forest



[eCommerce Customer Service Satisfaction](https://www.kaggle.com/datasets/ddosad/ecommerce-customer-service-satisfaction) data from Kaggle

#### 1. Import Libraries

In [ ]:
import pandas as pd
from plotnine import *
from datetime import datetime as dt
import re

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

#### 2. Import Data

In [9]:
cust_satisfaction_df = pd.read_csv('Customer_support_data.csv')

#### 3. Exploratory Analysis

In [43]:
cust_satisfaction_df.head()

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,...,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score,order_date,issue_date,response_date
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,01/08/2023 11:13,01/08/2023 11:47,01-Aug-23,...,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5,NaT,2023-08-01,2023-08-01
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,01/08/2023 12:52,01/08/2023 12:54,01-Aug-23,...,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5,NaT,2023-08-01,2023-08-01
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,01/08/2023 20:16,01/08/2023 20:38,01-Aug-23,...,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5,NaT,2023-08-01,2023-08-01
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,01/08/2023 20:56,01/08/2023 21:16,01-Aug-23,...,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5,NaT,2023-08-01,2023-08-01
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,01/08/2023 10:30,01/08/2023 10:32,01-Aug-23,...,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5,NaT,2023-08-01,2023-08-01


In [44]:
cust_satisfaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Unique id                85907 non-null  object        
 1   channel_name             85907 non-null  object        
 2   category                 85907 non-null  object        
 3   Sub-category             85907 non-null  object        
 4   Customer Remarks         28742 non-null  object        
 5   Order_id                 67675 non-null  object        
 6   order_date_time          17214 non-null  object        
 7   Issue_reported at        85907 non-null  object        
 8   issue_responded          85907 non-null  object        
 9   Survey_response_Date     85907 non-null  object        
 10  Customer_City            17079 non-null  object        
 11  Product_category         17196 non-null  object        
 12  Item_price               17206 n

In [ ]:
cust_satisfaction_df['order_date'] = cust_satisfaction_df['order_date_time'].apply(lambda x: dt.strptime(str(x)[:10],'%d/%m/%Y') if not pd.isna(x) else x).copy()
cust_satisfaction_df['response_date'] = cust_satisfaction_df['issue_responded'].apply(lambda x: dt.strptime(str(x)[:10],'%d/%m/%Y') if not pd.isna(x) else x).copy()
cust_satisfaction_df['issue_date'] = cust_satisfaction_df['Issue_reported at'].apply(lambda x: dt.strptime(str(x)[:10],'%d/%m/%Y') if not pd.isna(x) else x).copy()

In [14]:
cust_satisfaction_df['category'].value_counts()

category
Returns               44097
Order Related         23215
Refund Related         4550
Product Queries        3692
Shopzilla Related      2792
Payments related       2327
Feedback               2294
Cancellation           2212
Offers & Cashback       480
Others                   99
App/website              84
Onboarding related       65
Name: count, dtype: int64

#### 4. Machine Learning

For the machine learning portion, we are going to build a decision tree to predict the CSAT Score. Before we do this, we'll need to do some pre-processing to develop a data set that is best equipped to handle this request

In [46]:
cust_satisfaction_df.head()

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,...,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score,order_date,issue_date,response_date
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,01/08/2023 11:13,01/08/2023 11:47,01-Aug-23,...,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5,NaT,2023-08-01,2023-08-01
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,01/08/2023 12:52,01/08/2023 12:54,01-Aug-23,...,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5,NaT,2023-08-01,2023-08-01
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,01/08/2023 20:16,01/08/2023 20:38,01-Aug-23,...,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5,NaT,2023-08-01,2023-08-01
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,01/08/2023 20:56,01/08/2023 21:16,01-Aug-23,...,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5,NaT,2023-08-01,2023-08-01
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,01/08/2023 10:30,01/08/2023 10:32,01-Aug-23,...,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5,NaT,2023-08-01,2023-08-01


In [ ]:
pd.DataFrame(cust_satisfaction_df.columns)

,0
0,Unique id
1,channel_name
2,category
3,Sub-category
4,Customer Remarks
5,Order_id
6,order_date_time
7,Issue_reported at
8,issue_responded
9,Survey_response_Date


In [ ]:
keep_cols = [1,2,3,10,11,12,14,15,16,17,20,21,22,19]

In [ ]:
csat_df = cust_satisfaction_df.iloc[:,keep_cols]

In [ ]:
csat_df.head(5)

,channel_name,category,Sub-category,Customer_City,Product_category,Item_price,Agent_name,Supervisor,Manager,Tenure Bucket,order_date,issue_date,response_date,CSAT Score
0,Outcall,Product Queries,Life Insurance,NaN,NaN,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,NaT,2023-08-01,2023-08-01,5
1,Outcall,Product Queries,Product Specific Information,NaN,NaN,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,NaT,2023-08-01,2023-08-01,5
2,Inbound,Order Related,Installation/demo,NaN,NaN,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,NaT,2023-08-01,2023-08-01,5
3,Inbound,Returns,Reverse Pickup Enquiry,NaN,NaN,NaN,Patrick Flores,Olivia Wang,John Smith,>90,NaT,2023-08-01,2023-08-01,5
4,Inbound,Cancellation,Not Needed,NaN,NaN,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,NaT,2023-08-01,2023-08-01,5


Make additional changes to the data to clean up the available features

1. Customer City --> Remove this
2. Item Price --> Binned value
3. Tenure Bucket --> Change to categorical
4. Orer Date --> Remove this
4. CSAT Score --> Change to categorical
5. Create a time to response variable based on issue_date and response_date variable

In [101]:
mod_df = csat_df.loc[:,~csat_df.columns.isin(['Customer_City','Item_price','order_date'])].copy()

In [57]:
mod_df.head()

,channel_name,category,Sub-category,Product_category,Agent_name,Supervisor,Manager,Tenure Bucket,issue_date,response_date,CSAT Score
0,Outcall,Product Queries,Life Insurance,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,2023-08-01,2023-08-01,5
1,Outcall,Product Queries,Product Specific Information,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,2023-08-01,2023-08-01,5
2,Inbound,Order Related,Installation/demo,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,2023-08-01,2023-08-01,5
3,Inbound,Returns,Reverse Pickup Enquiry,NaN,Patrick Flores,Olivia Wang,John Smith,>90,2023-08-01,2023-08-01,5
4,Inbound,Cancellation,Not Needed,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,2023-08-01,2023-08-01,5


In [103]:
col_names = [col.lower() for col in mod_df.columns]

In [104]:
col_names = [re.sub('[-\s]',"_",col) for col in col_names]

In [105]:
mod_df = mod_df.set_axis(col_names, axis='columns')

In [107]:
mod_df.product_category.fillna("Other",inplace=True)

In [110]:
mod_df.product_category.value_counts()/85907

product_category
Other                          0.799830
Electronics                    0.054780
LifeStyle                      0.047936
Books & General merchandise    0.038681
Mobile                         0.020464
Home                           0.015459
Home Appliences                0.015133
Furniture                      0.005483
Affiliates                     0.001932
GiftCard                       0.000303
Name: count, dtype: float64

Given the large number of NA Values in product category, I'm going to remove that columns from the data

In [115]:
mod_df = mod_df.loc[:,~mod_df.columns.isin(['product_category'])].copy()

In [122]:
mod_df['response_days'] = mod_df.apply(lambda x: (x['response_date']-x['issue_date']).days,axis=1)

In [125]:
mod_df = mod_df.loc[:,~mod_df.columns.isin(['issue_date','response_date'])].copy()

In [128]:
mod_df['csat_score'] = pd.Categorical(mod_df['csat_score'])

In [129]:
mod_df

,channel_name,category,sub_category,agent_name,supervisor,manager,tenure_bucket,csat_score,response_days
0,Outcall,Product Queries,Life Insurance,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,5,0
1,Outcall,Product Queries,Product Specific Information,Vicki Collins,Dylan Kim,Michael Lee,>90,5,0
2,Inbound,Order Related,Installation/demo,Duane Norman,Jackson Park,William Kim,On Job Training,5,0
3,Inbound,Returns,Reverse Pickup Enquiry,Patrick Flores,Olivia Wang,John Smith,>90,5,0
4,Inbound,Cancellation,Not Needed,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,5,0
...,...,...,...,...,...,...,...,...,...
85902,Inbound,Refund Related,Refund Enquiry,Brandon Leon,Ethan Tan,William Kim,On Job Training,4,1
85903,Inbound,Order Related,Seller Cancelled Order,Linda Foster,Noah Patel,Emily Chen,>90,5,0
85904,Inbound,Order Related,Order status enquiry,Kimberly Martinez,Aiden Patel,Olivia Tan,On Job Training,5,0
85905,Inbound,Feedback,UnProfessional Behaviour,Daniel Martin,Olivia Suzuki,Olivia Tan,>90,4,0


#### 4.1 Decision Tree #1

In [161]:
X1 = mod_df.iloc[:,~mod_df.columns.isin(['csat_score'])]

In [162]:
y = mod_df['csat_score']

In [163]:
X1_dummies = pd.get_dummies(X1,dtype='int')

In [165]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1_dummies, y,test_size=0.7)

In [166]:
clf1 = DecisionTreeClassifier()

In [168]:
clf1.fit(X1_train,y1_train)

DecisionTreeClassifier()

In [169]:
clf1.score(X1_test, y1_test)

0.5809096200216181

#### 4.2 Decision Tree #2

In [170]:
X2 = mod_df.iloc[:,~mod_df.columns.isin(['csat_score','response_days'])]

In [171]:
y2 = mod_df['csat_score']

In [172]:
X2_dummies = pd.get_dummies(X2,dtype='int')

In [173]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_dummies, y2,test_size=0.7)

In [155]:
clf2 = DecisionTreeClassifier()

In [174]:
clf2.fit(X2_train,y2_train)

DecisionTreeClassifier()

In [176]:
clf2.score(X2_test, y2_test)

0.576768936559408

#### 4.3 Random Forest

In [178]:
clf3 = RandomForestClassifier(n_estimators = 50)
clf4 = RandomForestClassifier(n_estimators = 50)

In [179]:
clf3.fit(X1_train, y1_train)
clf4.fit(X2_train, y2_train)

RandomForestClassifier(n_estimators=50)

In [160]:
clf3.score(X1_test, y1_test)


0.6340234472437016

In [180]:
clf4.score(X2_test, y2_test)

0.6350378315456888

#### 5. Conclusion